In [35]:
import json
import boto3
import psycopg2
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import os
from datetime import datetime, timedelta
import pandas as pd

def lambda_handler(event, context):
    # Fetch data from NBA API
    yesterday = datetime.now() - timedelta(days=1)
    yesterday_str = yesterday.strftime('%m/%d/%Y')

    gamefinder = leaguegamefinder.LeagueGameFinder(
        league_id_nullable='00',            # '00' corresponds to the NBA
        season_nullable='2024-25',
        date_to_nullable=yesterday_str
    )
    games = gamefinder.get_data_frames()[0]
    
    return games

def process_games(df):
    # Filter for regular season games
    df = df[df['SEASON_ID'].str.startswith('2')]
    
    # Fix Dates
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    
    
    return df

# Test
event = {}
context = {} 
games = lambda_handler(event, context)
games_clean = process_games(games)

SEASON_ID                    object
TEAM_ID                       int64
TEAM_ABBREVIATION            object
TEAM_NAME                    object
GAME_ID                      object
GAME_DATE            datetime64[ns]
MATCHUP                      object
WL                           object
MIN                           int64
PTS                           int64
FGM                           int64
FGA                           int64
FG_PCT                      float64
FG3M                          int64
FG3A                          int64
FG3_PCT                     float64
FTM                           int64
FTA                           int64
FT_PCT                      float64
OREB                          int64
DREB                          int64
REB                           int64
AST                           int64
STL                           int64
BLK                           int64
TOV                           int64
PF                            int64
PLUS_MINUS                  

C:\Users\jorda\AppData\Local\Temp\ipykernel_9812\3696120194.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612755,PHI,Philadelphia 76ers,0022400050,2024-12-03,PHI @ CHA,W,241,110,...,0.900,8,23,31,26,9,3,12,22,6.0
1,22024,1610612758,SAC,Sacramento Kings,0022400059,2024-12-03,SAC vs. HOU,W,241,120,...,0.600,10,24,34,33,7,8,10,20,9.0
2,22024,1610612742,DAL,Dallas Mavericks,0022400056,2024-12-03,DAL vs. MEM,W,239,121,...,0.682,14,35,49,24,8,8,25,23,5.0
3,22024,1610612744,GSW,Golden State Warriors,0022400058,2024-12-03,GSW @ DEN,L,241,115,...,0.714,11,37,48,36,12,3,14,25,-4.0
4,22024,1610612761,TOR,Toronto Raptors,0022400054,2024-12-03,TOR vs. IND,W,241,122,...,0.724,6,36,42,28,9,4,21,26,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,22024,1610612766,CHA,Charlotte Hornets,0022400068,2024-10-23,CHA @ HOU,W,241,110,...,0.792,15,41,56,20,6,10,16,21,5.0
632,22024,1610612738,BOS,Boston Celtics,0022400061,2024-10-22,BOS vs. NYK,W,240,132,...,0.875,11,29,40,33,6,3,3,15,23.0
633,22024,1610612747,LAL,Los Angeles Lakers,0022400062,2024-10-22,LAL vs. MIN,W,240,110,...,0.840,15,31,46,22,7,8,7,22,7.0
634,22024,1610612750,MIN,Minnesota Timberwolves,0022400062,2024-10-22,MIN @ LAL,L,239,103,...,0.741,12,35,47,17,4,1,15,22,-7.0


In [6]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')